In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import typing
import math

pd.set_option("display.max_columns", None)

In [2]:
class Processor(pd.DataFrame):
    """用于预处理数据的数据框类,数据格式应为: 前两列分别为股票代码和时间的索引,后续列为数据
    
    asset / symbol_id:              字符串;类型,形如"0051"
    datetime / trade_date:          字符串类型
    value:                          float类型
    """

    def __init__(self, *args, **kwargs):
        super(Processor, self).__init__(*args, **kwargs)

    def filter_asset(self,asset_pool:pd.DataFrame):
        """用asset_pool股票池中的股票代码对数据中的股票代码进行筛选
        
        asset_pool:     首列为股票代码的数据集
        """

        asset_pool = asset_pool.iloc[:,0:1].rename(columns = {asset_pool.columns[0]:"asset"})
        asset_pool['asset'] = asset_pool['asset'].apply(lambda x: str(x))
        self = pd.merge(self,asset_pool,how='inner',on = 'asset').sort_values(['asset','datetime'])

        return Processor(self)

    def drop_samevalue(self):
        """除去前两列,若数据某一列值全部相同或全为Na则将该列丢掉"""

        drop_list = [column for column in self.columns[2:] if not self[column].std() > 0]

        return Processor(self.drop(columns = drop_list))

    def fill_na(self):
        """填充数据中的缺失值,填充方式为该列均值,该列全部缺失则填充为0"""

        for i in range(2,len(self.columns)):
            mean_value = self.iloc[:,i].mean()
            self[self.columns[i]] = self[self.columns[i]].fillna(mean_value)
        
        assert self.isnull().all().sum() == 0, "错误:存在某一列因子值全部为Na导致无法填充"

        return self
    
    def rescaling(self,method = "max_min",extreme_value_replace = False):
        """进行去极值,然后再进行数据归一化或标准化处理,method可选max_min,z_score,ranknorm"""

        # 中位数去极值也会出现问题,例如某个因子序列为[0,1,1,1,1,1],则median为1,median2为0,
        # 此时限制区间变为[1,1],导致因子只能被修改为同一个值即1,故改用(mean-2sigma,mean+2sigma)
        # if extreme_value_replace == True:                   #中位数去极值,将因子值限制在(median-5*median2, median+5*median2)之内
        #   for i in range(2,len(self.columns)):
        #     median_value = self.iloc[:,i].median()
        #     median_value2 = np.median(np.abs(self.iloc[:,i]-median_value))
        #     self[self.columns[i]] = np.clip(self[self.columns[i]], median_value-5*median_value2, median_value+5*median_value2)

        if extreme_value_replace == True:                   #用均值和方差去极值,将因子值限制在(mean-2sigma,mean+2sigma)范围内
            for i in range(2,len(self.columns)):
                mean_value = self.iloc[:,i].mean()
                sigma_value = self.iloc[:,i].std()
                self[self.columns[i]] = np.clip(self[self.columns[i]], mean_value-2*sigma_value, mean_value+2*sigma_value)

        if method == "max_min":                             #选择归一化或标准化方法
            for i in range(2,len(self.columns)):
                max_value = self.iloc[:,i].max()
                min_value = self.iloc[:,i].min()
                self[self.columns[i]] = (self[self.columns[i]] - min_value)/(max_value - min_value)
        
        elif method == "z_score":
            for i in range(2,len(self.columns)):
                mean_value = self.iloc[:,i].mean()
                sigma_value = self.iloc[:,i].std()
                self[self.columns[i]] = (self[self.columns[i]] - mean_value)/sigma_value

        elif method == "ranknorm":
            for i in range(2,len(self.columns)):
                self[self.columns[i]] = self[self.columns[i]].rank(pct = True)                

        else:
            raise Exception("method参数选择错误")

        assert (self.iloc[:,2:].std() > 0).sum() == len(self.iloc[:,2:].columns), "错误:存在某列因子值全部相同或全部为Na"

        return self

    def change_shape(self,unstack = True):
        """若数据频率不高于1d,则可以将一天内m个时间段的n种因子的值更改为1*mn的形式
        
        unstack:    为True指将m个时间段的n种因子的值从m*n的形式更改为1*mn形式
                    为False指将1*mn形式更改回m*n形式
        """
        
        if unstack:
            try:
                self = self.sort_values(["asset","datetime"])
                self['minute'] = self['datetime'].apply(lambda x: x[-8:])
                self['datetime'] = self['datetime'].apply(lambda x: x[:10])
            except:
                raise Exception("数据中的datetime列不适用于reshape或数据已达到所需shape")

            self = self.sort_values(['asset','datetime','minute']).set_index(['asset','datetime','minute']).unstack(2)
            self.columns = ["_".join(tuple) for tuple in self.columns]
            self = self.reset_index()

        else:
            try:
                self = self.sort_values(["asset","datetime"]).set_index(["asset","datetime"])
                column_list = ["asset","datetime"] + list(dict.fromkeys([column[:-9] for column in self.columns]))
                self.columns = pd.MultiIndex.from_tuples([(column[:-9],column[-8:]) for column in self.columns],names = ("","minute"))
                self = self.stack(1).reset_index()
            except:
                raise Exception("数据中的列名不适用于reshape或数据已达到所需shape")

            self["datetime"] = self["datetime"].str.cat(self["minute"],sep = " ")
            self = self.drop(columns = ["minute"])[column_list]
            
        return Processor(self)
    
    def dataprocessing(self,asset_pool,method = "max_min",extreme_value_replace = False,unstack = True,):
        """使用上述方法进行一个完整的数据处理过程"""
        
        self = self.filter_asset(asset_pool)
        self = self.change_shape(unstack = True)
        self = self.drop_samevalue()
        self = self.fill_na()
        self = self.rescaling(method, extreme_value_replace)

        
        return Processor(self)

In [3]:
class LabelDataFrame(Processor):
    """用于处理标签数据的数据框类,数据格式应为含有trade_date、symbol_id和交易价格的price数据
    
    trade_date:         字符串类型,形如"20100104"
    symbol_id:          字符串类型,形如"0015"
    close_price:        float类型
    adj_close_price:    float类型
    """

    def __init__(self, *args, **kwargs):
        super(LabelDataFrame, self).__init__(*args, **kwargs)

    def get_return(self, adj = False, return_frequency = "1d", close_open = False):
        """对数据按代码和时间进行排序,增添新列为计算出的收益率,列名为return,收益率为na则填充为0
        
        adj:                    选择是否用复权价格计算,默认为False
        return_frequency:       选择收益率计算的天数,可选"1d","2d","1m","2m"等,默认为"1d"
        close_open:             选择计算收益率是否采用close-open的方式,默认为False,为True时只计算当天
        """

        adj_str = adj and "adj_" or ""                                    #选择是否在列名前加上adj_
        self = self.sort_values(["symbol_id","trade_date"])
        fre_num = int(return_frequency[:-1])

        if close_open:
            self["return"] = np.log( self[adj_str+"close_price"]/self[adj_str+"open_price"] )
        else:
            if return_frequency[-1] == "d":
                self["return"] = np.log( self[["symbol_id",adj_str+"close_price"]].groupby("symbol_id")[adj_str+"close_price"].shift(1-fre_num)/\
                                            self[["symbol_id",adj_str+"close_price"]].groupby("symbol_id")[adj_str+"close_price"].shift(1) )
            elif return_frequency[-1] == "m":
                self["year_month"] = self["trade_date"].apply(lambda x: x[0:6])
                temp_df = self[["symbol_id",adj_str+"close_price","year_month"]].groupby(["symbol_id","year_month"]).last().reset_index()
                temp_df["return"] = np.log( temp_df.groupby("symbol_id")[adj_str+"close_price"].shift(1-fre_num)/\
                                             temp_df.groupby("symbol_id")[adj_str+"close_price"].shift(1) )
                self = pd.merge(self,temp_df[["symbol_id","year_month","return"]], how = "left", on = ["symbol_id","year_month"])          
                self = self.drop(columns = ["year_month"]) 
            else:
                raise Exception("return_frequency值选择错误")                   

        self["return"] = self["return"].fillna(0)

        return LabelDataFrame(self)

In [4]:
class MergeData:
    """特征数据与标签数据的合并类"""
    
    def __init__(self,featuredata:Processor,labeldata:LabelDataFrame):
        self.featuredata = featuredata
        self.labeldata = labeldata
        self.mergedata = pd.DataFrame()

    def merge_feature_label(self):
        """将mergedata属性更改为特征数据与标签数据合并后的数据框
        
        合并后mergedata的格式为:前两列分别为asset和datetime作为索引,中间为特征数据,最后一列return为标签数据
        """
        
        df_feature_temp, df_label_temp = self.featuredata.copy(), self.labeldata.copy()
        df_feature_temp['trade_date'] = df_feature_temp["datetime"].apply(lambda x: x[:10].replace("-",""))
        df_feature_temp['symbol_id'] = df_feature_temp["asset"]

        self.mergedata = pd.merge(df_feature_temp,df_label_temp[["trade_date","symbol_id","return"]],how = "inner",on = ["trade_date","symbol_id"])
        self.mergedata = self.mergedata.drop(columns = ["trade_date","symbol_id"]).reset_index(drop = True)

        return self

    def split_index_feature_label(self) -> list:
        """将合并后的数据拆分为索引,特征数据,标签数据,并返回三者,此时三者是对齐的"""
        
        if len(self.mergedata) != 0:
            df_index = self.mergedata.iloc[:,:2]
            df_feature = self.mergedata.iloc[:,2:-1]
            df_label = self.mergedata.iloc[:,-1]
        else:
            df_index, df_feature, df_label = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
        
        return df_index, df_feature, df_label

In [5]:
if __name__ == "__main__":
    """数据预处理步骤如下:

    1.特征数据:股票池筛选,reshape,问题列丢弃,na值填充,去极值,归一化
    2.股价数据:计算收益率标签,标签去极值和归一化
    3.特征数据与标签数据merge,对齐索引后拆分为索引,特征,标签三个数据,保存
    """

    path_dict = {"asset_pool_path":"/sda/intern_data_shuofu/Data/industry_1070.csv",
                 "feature_path": "/home/qianshuofu/factor_qianshuofu/Data/30_minutes_data.feather",
                 "label_path": "/home/qianshuofu/factor_qianshuofu/Data/adj_prices.feather",

                 "index_save_path": "/home/qianshuofu/factor_qianshuofu/Data/data_index.npy",
                 "feature_save_path": "/home/qianshuofu/factor_qianshuofu/Data/data_feature.npy",
                 "label_save_path": "/home/qianshuofu/factor_qianshuofu/Data/data_label.npy"
    }

    asset_pool = pd.read_csv(path_dict["asset_pool_path"])
    data_feature = Processor(pd.read_feather(path_dict["feature_path"])).dataprocessing(asset_pool,extreme_value_replace=True)
    data_label = LabelDataFrame(pd.read_feather(path_dict['label_path'])).get_return()
    data_label = LabelDataFrame(data_label.iloc[:,[0,1,-1]]).rescaling(extreme_value_replace=True)

    data_merge = MergeData(data_feature,data_label).merge_feature_label()
    df_index, df_feature, df_label = data_merge.split_index_feature_label()

    np.save(path_dict["index_save_path"], df_index)
    np.save(path_dict["feature_save_path"], df_feature)
    np.save(path_dict["label_save_path"], df_label)


In [12]:
df_feature

up_over_down_for_vol_09:30:00  up_over_down_for_vol_10:00:00  \
0                            0.507382                       0.352527   
1                            0.127283                       0.549084   
2                            0.532389                       0.675502   
3                            0.272727                       0.657645   
4                            0.504598                       0.571416   
...                               ...                            ...   
911982                       0.050981                       0.310083   
911983                       0.816677                       0.932854   
911984                       0.404927                       0.295073   
911985                       0.238479                       0.494514   
911986                       0.113889                       0.624073   

        up_over_down_for_vol_10:30:00  up_over_down_for_vol_11:00:00  \
0                            0.758548                       1.000000   
1                            0.199239                       0.432092   
2                            0.430236                       0.502876   
3                            0.453220                       0.645906   
4                            0.420767                       0.729278   
...                               ...                            ...   
911982                       0.508202                       0.047381   
911983                       0.819940                       0.688846   
911984                       0.124570                       0.680596   
911985                       0.219799                       0.229461   
911986                       0.498897                       0.274045   

        up_over_down_for_vol_11:30:00  up_over_down_for_vol_12:00:00  \
0                            0.857733                       0.543154   
1                            0.569604                       0.554506   
2                            0.619068                       0.508616   
3                            0.460677                       0.457908   
4                            0.543312                       0.863994   
...                               ...                            ...   
911982                       0.228276                       0.599820   
911983                       0.641918                       0.574368   
911984                       0.275842                       0.293542   
911985                       0.180588                       0.707130   
911986                       0.622542                       0.827615   

        up_over_down_for_vol_12:30:00  up_over_down_for_vol_13:00:00  \
0                            0.780751                       0.607399   
1                            0.406179                       0.552862   
2                            0.573600                       0.647597   
3                            0.566006                       0.526188   
4                            0.494075                       0.641234   
...                               ...                            ...   
911982                       0.232002                       0.280017   
911983                       0.814377                       0.867312   
911984                       0.417189                       0.212040   
911985                       0.565760                       0.247260   
911986                       0.526106                       0.658192   

        up_over_down_for_vol_13:30:00  trace_09:30:00  trace_10:00:00  \
0                            0.467395        0.323980        0.340322   
1                            0.459217        0.400472        0.385131   
2                            0.075764        0.275472        0.157938   
3                            0.215227        0.246117        0.294831   
4                            0.829193        0.245326        0.588169   
...                               ...             ...             ...   
911982                       0.089804